# Training

### Importing necessary libraries

In [1]:
import numpy as np 
import tensorflow as tf
from tensorflow import keras, image
from keras import layers, models, Input
from keras.layers import Dense, Dropout, Flatten, Reshape, Resizing, BatchNormalization, Activation, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Normalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications.resnet_v2 import ResNet50V2

#random seed
tf.random.set_seed(0)

### Model architecture

In [2]:
# Creating model architecture

base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#making all layers in resnet trainable
for layer in base_model.layers[:-50]:
    layer.trainable = False

model = models.Sequential([
    Normalization(mean=[0.485, 0.456, 0.406], variance=[0.229, 0.224, 0.225], input_shape=(300, 300, 3)),
    Resizing(224, 224),
    base_model,
    Dropout(0.5),
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(512, activation='relu'),
    Dropout(0.25),
    BatchNormalization(),
    Dense(10, activation='softmax')
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

2023-04-25 18:30:12.983227: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 18:30:13.582508: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79111 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0


### Train function

In [3]:
### Function to train the model

def train(data, labels, model):

    #loading the data
    data = np.load(data)
    labels = np.load(labels)

    #Transposing the data matrix to align dimensions for reshaping
    data = data.T

    #reshaping the data matrix into images
    num_images = data.shape[0]
    num_channels = 3

    #empty matrix to store reshaped images
    images = np.zeros((num_images, 300, 300, num_channels), dtype=np.float32)

    for i in range(num_images):
        image = data[i,:].reshape((300, 300, num_channels))
        #standardizing the images by dividing by 255
        image = image/255.0
        images[i] = image

    #done with data array, deleting to free up memory
    del data

    #validation split

    images, val_images, labels, val_labels = train_test_split(images, labels, test_size=0.1, shuffle=True, random_state=0)

    #callback checkpoint
    checkpoint_cb = keras.callbacks.ModelCheckpoint('model.h5', 
                                                save_best_only=True)

    #fitting the model

    history = model.fit(images, labels, epochs=50, batch_size=32, validation_data=(val_images, val_labels), callbacks=[checkpoint_cb])

    #returning the history object

    return history

### Running function. Replace "data_train.npy" and "labels_train.npy", with the file name for the data and labels that you wish to train with. Include the quotation marks.

In [4]:
train("data_train.npy", "labels_train.npy", model)

Epoch 1/50


2023-04-25 18:30:24.156301: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2023-04-25 18:30:25.285784: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Running ptxas --version returned 32512
2023-04-25 18:30:25.511380: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: ptxas exited with non-zero error code 32512, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-04-25 18:30:26.654995: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


105/105 [==============================] - ETA: 0s - loss: 0.8305 - accuracy: 0.7776

/apps/tensorflow/2.7.0/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


105/105 [==============================] - 11s 48ms/step - loss: 0.8305 - accuracy: 0.7776 - val_loss: 17.9701 - val_accuracy: 0.3952
Epoch 2/50
105/105 [==============================] - 4s 34ms/step - loss: 0.3963 - accuracy: 0.9019 - val_loss: 0.6570 - val_accuracy: 0.8817
Epoch 3/50
105/105 [==============================] - 4s 35ms/step - loss: 0.3077 - accuracy: 0.9247 - val_loss: 0.6331 - val_accuracy: 0.8898
Epoch 4/50
105/105 [==============================] - 4s 34ms/step - loss: 0.2299 - accuracy: 0.9438 - val_loss: 0.4809 - val_accuracy: 0.9113
Epoch 5/50
105/105 [==============================] - 4s 34ms/step - loss: 0.1827 - accuracy: 0.9567 - val_loss: 0.3608 - val_accuracy: 0.9194
Epoch 6/50
105/105 [==============================] - 4s 34ms/step - loss: 0.1315 - accuracy: 0.9638 - val_loss: 0.3248 - val_accuracy: 0.9382
Epoch 7/50
105/105 [==============================] - 3s 29ms/step - loss: 0.1068 - accuracy: 0.9713 - val_loss: 0.4076 - val_accuracy: 0.9382
Epoch 8/